In [1]:
import torch
from slot_attention.model import SlotAttentionModel
from slot_attention.data import CLEVRERDataModule
from torchvision import transforms
from slot_attention.utils import rescale
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

In [2]:
clevr_transforms = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Lambda(rescale),
    transforms.Resize((160,240)),
    ])

clevr_datamodule = CLEVRERDataModule(data_root='data/CLEVRER_v2/',
                                   
                                   train_batch_size=1,
                                   val_batch_size=1,
                                   clevr_transforms=clevr_transforms,
                                   num_workers=4)

In [3]:
model = SlotAttentionModel(resolution=(160,240),num_slots=7,num_iterations=3).cuda()

In [4]:
state_dict = torch.load('wandb/run-20210312_164330-3sa2m0bw/files/slot-attention-clevrer/3sa2m0bw/checkpoints/epoch=499-step=112999.ckpt', map_location=lambda storage, loc: storage)['state_dict']
from collections import OrderedDict
state_dict_new = OrderedDict()
for k, v in state_dict.items():
    name = k[6:]  # 去掉 `model.`
    state_dict_new[name] = v

model.load_state_dict(state_dict_new)

<All keys matched successfully>

In [5]:
dl = clevr_datamodule.val_dataloader()

In [7]:
batch = next(iter(dl))
recon_combined, recons, masks, slots, recon_combined_preds, recons_preds, masks_preds, slots_preds = model(batch.cpu())
import numpy as np
from PIL import Image
display(Image.fromarray(((recon_combined[0]*0.5+0.5).clamp(0, 1).detach().cpu().numpy()*255).astype(np.uint8).transpose(1,2,0)))
display(Image.fromarray(((recons[0][0]*0.5+0.5).clamp(0, 1).detach().cpu().numpy()*255).astype(np.uint8).transpose(1,2,0)))

RuntimeError: CUDA out of memory. Tried to allocate 264.00 MiB (GPU 0; 11.78 GiB total capacity; 10.16 GiB already allocated; 149.69 MiB free; 10.32 GiB reserved in total by PyTorch)

In [ ]:

for i in range(64):
    for j in [-1,0,1]:
        slots_f = slots.clone()
        slots_f[0][i]=j
        recon_combined, recons, masks, _ = model.forward(batch,force_slots=slots_f)
        print(i,j)
        #isplay(Image.fromarray(((recon_combined[0]*0.5+0.5).clamp(0, 1).detach().numpy()*255).astype(np.uint8).transpose(1,2,0)))
        display(Image.fromarray(((recons[0][0]*0.5+0.5).clamp(0, 1).detach().numpy()*255).astype(np.uint8).transpose(1,2,0)))

In [ ]:
slots[0]